<a href="https://colab.research.google.com/github/A-stell-A/weatherwise-template/blob/main/Weatherwise_Part1_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pydantic-ai==0.1.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.7/152.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.0/264.0 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.5/301.5 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 107.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [14]:
import nest_asyncio
nest_asyncio.apply()

**Weather Forecast - Function Tool**

Custom Weather Forecast Tool - OWM

Get API keys from: https://home.openweathermap.org/api_keys


In [1]:
import requests

BASE_URL = "https://api.openweathermap.org/data/2.5/weather"
API_KEY = "9e58001aecb184f42bbfe60f3ce9751e"

def find_weather(city: str) -> dict:
    """This function returns the current weather forecast for the given city"""
    units = "metric"
    params = {
        'q': city,
        'appid': API_KEY,
        'units': units
    }
    response = requests.get(BASE_URL, params=params)

    # Check if the response is valid JSON and status is OK
    if response.status_code == 200 and 'application/json' in response.headers.get('Content-Type', ''):
        return response.json()
    else:
        return {"error": f"Request failed. Status: {response.status_code}", "content": response.text}


In [2]:
op = find_weather("chennai")
print(op)

{'coord': {'lon': 80.2785, 'lat': 13.0878}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d'}], 'base': 'stations', 'main': {'temp': 32.03, 'feels_like': 39.03, 'temp_min': 31.67, 'temp_max': 32.25, 'pressure': 1007, 'humidity': 72, 'sea_level': 1007, 'grnd_level': 1007}, 'visibility': 10000, 'wind': {'speed': 2.14, 'deg': 196, 'gust': 2.73}, 'clouds': {'all': 47}, 'dt': 1747365182, 'sys': {'type': 2, 'id': 2094849, 'country': 'IN', 'sunrise': 1747354396, 'sunset': 1747400246}, 'timezone': 19800, 'id': 1264527, 'name': 'Chennai', 'cod': 200}


**Pydantic Agent**

In [3]:
!pip install requests pydantic pydantic-ai

In [4]:
import os
import requests
from pydantic import BaseModel
from pydantic_ai import Agent, RunContext
from pydantic_ai.settings import ModelSettings

In [5]:
os.environ["GROQ_API_KEY"] = "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"

In [6]:
# Define the output schema
class WeatherForecast(BaseModel):
  location: str
  description: str
  temperature_cellsius: float

In [10]:
weather_agent = Agent(
    model="groq:llama-3.1-8b-instant",
    model_settings=ModelSettings(temperature=0.2),
    output_type=str,
    system_prompt=(
        "You are a helpful weather assistant. Use the 'get_weather_forecast' tool to find current weather "
        "conditions for any city. Provide clean and friendly answers."
    )
)

In [11]:
# weather forecast tool
@weather_agent.tool
def get_weather_forecast(ctx: RunContext, city: str) -> WeatherForecast:
  """
  Returns current weather forecast for a city using OpenWeatherMap API.
  """
  url = "https://openweathermap.org/weathermap?basemap=map&cities=true&layer=temperature&lat=42.6824&lon=-33.9697&zoom=5"
  api_key = "9e58001aecb184f42bbfe60f3ce9751e"
  params = {
      'q': city,
      'appid': api_key,
      'units': 'metric'
  }

  res = requests.get(url, params=params).json()

  return WeatherForecast(
      location=res["name"],
      description=res["weather"][0]["description"].capitalize(),
      temperature_celsius=res["main"]["temp"]
  )

In [15]:
question = input("Ask about the weather: ")

async def get_weather():
    result = await weather_agent.run(question)  # run() is async
    print("\nForecast:", result.output if hasattr(result, "output") else result)

await get_weather()

Ask about the weather: chennai

Forecast: <function=get_weather_forecast>{"city": "chennai"}
